In [ ]:
import pandas as pd
import numpy as np
import itertools
from scipy.stats import fisher_exact
import shelve
import pickle
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from PIL import Image
from scipy.interpolate import BSpline, make_interp_spline, interp1d
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
import csv
from dfply import *
import itertools
import os
import logging
from sys import getsizeof
import sklearn
import time
from sklearn.metrics import roc_auc_score
from catboost import Pool, cv
import xgboost
import catboost
import scipy.stats as st

In [ ]:
# Test Site
#sites = ['IUR', 'MCRI', 'MCW', 'UIOWA', 'UMHC', 'UNMC', 'UofU', 'UPITT', 'UTHSCSA', 'UTSW']
sites = ['IUR', 'MCRI', 'MCW', 'UIOWA', 'UMHC', 'UNMC', 'UofU', 'UPITT', 'UTHSCSA', 'UTSW']
site='UPITT'
year=3000
stg = 'stg01'
oversample = 'raw'
fs = 'nofs'
recol = '005'
model_type = 'catd'

In [ ]:
stg = 'stg23'
fs = 'nofs'
oversample='raw'
model = 'catd'    
rmcol = '005'

In [ ]:
bt = pd.read_pickle('data/'+site+'/bt3_'+site+'_'+stg+'_'+str(year)+'.pkl')

In [ ]:
bt.drop_duplicates().shape

In [ ]:
bt.shape

In [ ]:
target_feature = 'MED:ATC:N02BE'
df_list = list()
for site in sites:
    try:
        bt = pd.read_pickle('data/'+site+'/bt3_'+site+'_'+stg+'_'+str(year)+'.pkl')
        #bt[[target_feature,'FLAG']]
        base = (bt['FLAG']==1).sum()/bt.shape[0]
        support = bt[target_feature].sum()/bt.shape[0]
        support1 = np.logical_and(bt[target_feature],bt['FLAG']==1).sum()/(bt['FLAG']==1).sum()        
        conf = np.logical_and(bt[target_feature],bt['FLAG']==1).sum()/bt[target_feature].sum()
        df_list.append(pd.DataFrame({'site':site, 'support':support, 'support1':support1,'base':base, 'conf':conf},index=[0]))
    except:
        pass
df = pd.concat(df_list)

In [ ]:
site = 'UNMC'
home_directory = "/home/hchan2/AKI/AKI_Python/"    
onset = pd.read_pickle('data/'+site+'/p0_onset_'+site+'.pkl')
amed = pd.read_pickle('data/'+site+'/p0_amed_'+site+'.pkl')
amed_rx = amed.loc[amed['MEDADMIN_TYPE'] == "RX"]
rxcui2atc_dtypes =  {"Rxcui": 'object', "ATC4th": 'object'}    
rxcui2atc = pd.read_csv(home_directory+"data/"+site+'/rxnorm_out_'+site+'.csv',sep=',', dtype=(rxcui2atc_dtypes)) >> rename(MEDADMIN_CODE=X.Rxcui)
amed_rx = amed_rx >> left_join(rxcui2atc, by='MEDADMIN_CODE')
#amed_rx['MEDADMIN_TYPE'] = amed_rx['MEDADMIN_TYPE'].where(amed_rx['ATC4th'].isnull(), 'ATC')
#amed_rx['MEDADMIN_CODE'] = amed_rx['MEDADMIN_CODE'].where(amed_rx['ATC4th'].isnull(), amed_rx['ATC4th'])

In [ ]:
amed_rx[amed_rx['ATC4th']=='B01AB'][['PATID', 'MEDADMIN_CODE','ATC4th']].drop_duplicates().drop('PATID',axis=1).groupby('MEDADMIN_CODE').count()
#amed_rx[amed_rx['ATC4th']=='B01AB'][['MEDADMIN_CODE','ATC4th']].groupby('MEDADMIN_CODE').count()
#amed_rx[amed_rx['ATC4th']=='B01AB'][['PATID']].drop_duplicates().count()
#amed_rx[['PATID']].drop_duplicates().count()
#onset[['PATID']].drop_duplicates().count()

In [ ]:
def correlation_calculation(site, year, stg, target='LAB::17861-6(mg/dL)', threshold = 8.6, max_row=10, drop='LAB'):
    bt = pd.read_pickle('data/'+site+'/bt3_'+site+'_'+stg+'_'+str(year)+'.pkl')

    r1 = bt[target] > 8.6
    r3 = bt[target] < 8.6

    corrdict1 = dict()
    corrdict3 = dict()
    for col in bt.columns:
        if 'MED' in col:
            corrdict1[col] = bt[r1]['LAB::17861-6(mg/dL)'].corr(bt[r1][col])
            corrdict3[col] = bt[r3]['LAB::17861-6(mg/dL)'].corr(bt[r3][col])    

    #Correlation with high Calcium
    df1 = pd.DataFrame(corrdict1, index=[0]).T.sort_values(0,ascending=True).reset_index()
    #df1['index'] = df1['index'].str.split(':',expand=True).loc[:,2].str.split('(',expand=True).loc[:,0]
    #df1['name'] = df1['index'].apply(lablonic2name)
    df_high_anti = df1[~df1['index'].str.contains(drop)].dropna().head(max_row)
    df_high_anti['site'] = site
    df_high_anti['stg'] = stg
    
    #Correlation with high Calcium
    df1 = pd.DataFrame(corrdict1, index=[0]).T.sort_values(0,ascending=False).reset_index()
    #df1['index'] = df1['index'].str.split(':',expand=True).loc[:,2].str.split('(',expand=True).loc[:,0]
    #df1['name'] = df1['index'].apply(lablonic2name)
    df_high_corr = df1[~df1['index'].str.contains(drop)].dropna().head(max_row)
    df_high_corr['site'] = site
    df_high_corr['stg'] = stg

    #Correlation with low Calcium
    df1 = pd.DataFrame(corrdict3, index=[0]).T.sort_values(0,ascending=True).reset_index()
    #df1['index'] = df1['index'].str.split(':',expand=True).loc[:,2].str.split('(',expand=True).loc[:,0]
    #df1['name'] = df1['index'].apply(lablonic2name)
    df_low_anti = df1[~df1['index'].str.contains(drop)].dropna().head(max_row)
    df_low_anti['site'] = site
    df_low_anti['stg'] = stg

    #Correlation with low Calcium
    df1 = pd.DataFrame(corrdict3, index=[0]).T.sort_values(0,ascending=False).reset_index()
    #df1['index'] = df1['index'].str.split(':',expand=True).loc[:,2].str.split('(',expand=True).loc[:,0]
    #df1['name'] = df1['index'].apply(lablonic2name)
    df_low_corr = df1[~df1['index'].str.contains(drop)].dropna().head(max_row)
    df_low_corr['site'] = site
    df_low_corr['stg'] = stg
    
    return [df_high_corr, df_low_corr, df_high_anti, df_low_anti]

In [ ]:
sites = ['IUR', 'MCRI', 'MCW', 'UIOWA', 'UMHC', 'UNMC', 'UofU', 'UPITT', 'UTHSCSA', 'UTSW']
#stgs = ['stg01', 'stg23']
stgs = ['stg01']

df_high_corr = list()
df_low_corr = list()
for site in sites:
    for stg in stgs:
        try:
            print(site, stg)
            dfhc, dflc, d1, d2 = correlation_calculation(site, 3000, stg, target='LAB::17861-6(mg/dL)', threshold = 8.6, max_row=10, drop='LAB')
            df_high_corr.append(dfhc.copy())
            df_low_corr.append(dflc.copy())
        except:
            pass
df_high_corr = pd.concat(df_high_corr)
df_low_corr = pd.concat(df_low_corr)
df_high_corr.columns = ['index', 'corr', 'site', 'stg']
df_low_corr.columns = ['index', 'corr', 'site', 'stg']

In [ ]:
df_high_corr['rank'] = df_high_corr.groupby(['site','stg'])['corr'].rank("first", ascending=False)
df_high_corr['index(corr)'] = df_high_corr['index']+'('+df_high_corr['corr'].round(3).astype(str)+')'
df_high_corr[['site', 'index(corr)', 'rank']].pivot(index='site', columns='rank', values='index(corr)').applymap(lambda x: x.split(':')[-1])

In [ ]:
df_low_corr['rank'] = df_low_corr.groupby(['site','stg'])['corr'].rank("first", ascending=False)
df_low_corr['index(corr)'] = df_low_corr['index']+'('+df_low_corr['corr'].round(3).astype(str)+')'
df_low_corr[['site', 'index(corr)', 'rank']].pivot(index='site', columns='rank', values='index(corr)').applymap(lambda x: x.split(':')[-1])

In [ ]:
df_low_corr.to_csv('df_low_corr.csv',index=False)

In [ ]:
#read data session
datafolder = '/home/hchan2/AKI/data/'
datatt = pd.read_csv(datafolder+site+'/raw/'+'AKI_VITAL'+'.'+'csv')
onset = pd.read_pickle('data/'+site+'/p0_onset_'+site+'.pkl')
newdf = pd.read_pickle('data/'+site+'/onset_'+site+'_'+str(year)+'.pkl')
vital = pd.read_pickle('data/'+site+'/p0_vital_'+site+'.pkl')
vital_tt = pd.read_pickle('data/'+site+'/vital_'+site+'_'+str(year)+'.pkl')
demo = pd.read_pickle('data/'+site+'/p0_demo_'+site+'.pkl')
demo_t = pd.read_pickle('data/'+site+'/demo_'+site+'_'+str(year)+'.pkl')
dx = pd.read_pickle('data/'+site+'/p0_dx_'+site+'.pkl')
dx_t = pd.read_pickle('data/'+site+'/dx_'+site+'_'+str(year)+'.pkl')
px = pd.read_pickle('data/'+site+'/p0_px_'+site+'.pkl')
px_t = pd.read_pickle('data/'+site+'/px_'+site+'_'+str(year)+'.pkl')
lab = pd.read_pickle('data/'+site+'/p0_lab_'+site+'.pkl')
labnum_tt = pd.read_pickle('data/'+site+'/labnum_'+site+'_'+str(year)+'.pkl')
labcat_tt = pd.read_pickle('data/'+site+'/labcat_'+site+'_'+str(year)+'.pkl')
amed = pd.read_pickle('data/'+site+'/p0_amed_'+site+'.pkl')
amed_tt = pd.read_pickle('data/'+site+'/amed_'+site+'_'+str(year)+'.pkl')
bt = pd.read_pickle('data/'+site+'/bt3_'+site+'_'+str(year)+'.pkl')

X_train = pd.read_pickle('data/'+site+'/X_train_'+site+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+'.pkl')
X_test =  pd.read_pickle('data/'+site+ '/X_test_'+site+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+'.pkl')
y_train = pd.read_pickle('data/'+site+'/y_train_'+site+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+'.pkl')
y_test =  pd.read_pickle('data/'+site+ '/y_test_'+site+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+'.pkl')
shap_data = pd.read_pickle('data/'+site+'/shapdata_'+model_type+'_'+site+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+'_005.pkl')

model = pickle.load(open('data/'+site+'/model_'+model_type+'_'+site+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+'.pkl', 'rb'))
result = pd.read_pickle('result.pkl')

In [ ]:
#Load statistics
stg = 'stg01'
fs = 'rmscrbun'
oversample='raw'
model = 'catd'    
rmcol = '005'
result = pd.read_pickle('./DEID_resultsplit_'+model+'_'+stg+'_'+fs+'_'+oversample+'_005.pkl')

In [ ]:
#result2 = result.loc[result['site'] == 'KUMC']
#result2 = result2.loc[result['year'] == 2016]
#result2[['site', 'year', 'Feature', 'rank', 'rank_abs_shap_max', 'rank_minmax_SHAP', 'rank_var_SHAP']].drop_duplicates().sort_values('rank_var_SHAP')

resultX = result.loc[:,['site', 'year', 'Feature', 'mean_val']].groupby(['site', 'year', 'Feature']).agg([np.mean]).reset_index()
resultX.columns = ['site', 'year', 'Feature', 'meanmean_val'] 
resultX['rank_meanmean_SHAP'] = resultX.groupby(['site', 'year'])['meanmean_val'].rank(method='min', ascending=False)

resultY = result.loc[:,['site', 'year', 'Feature', 'mean_imp']].groupby(['site', 'year', 'Feature']).agg([np.mean]).reset_index()
resultY.columns = ['site', 'year', 'Feature', 'absmean_val'] 
resultY['rank_absmean_SHAP'] = resultY.groupby(['site', 'year'])['absmean_val'].rank(method='min', ascending=False)

result = pd.merge(result, resultX, left_on=['site', 'year', 'Feature'], right_on=['site', 'year', 'Feature'], how='left')    
result = pd.merge(result, resultY, left_on=['site', 'year', 'Feature'], right_on=['site', 'year', 'Feature'], how='left')    
result

In [ ]:
result.to_pickle('./DEID_resultsplit_'+model+'_'+stg+'_'+fs+'_'+oversample+'_005_2.pkl')

In [ ]:
result.columns

In [ ]:
result2 = result.loc[result['site'] == 'KUMC']
result2 = result2.loc[result['year'] == 2016]
result2[['site', 'year', 'Feature', 'rank', 'rank_abs_shap_max', 'rank_minmax_SHAP', 'rank_var_SHAP', 'rank_meanmean_SHAP', 'rank_absmean_SHAP']].drop_duplicates().sort_values('rank').head(10)

In [ ]:
stg = 'stg01'
fs = 'rmscrbun'
oversample='raw'
model_type = 'catd'    
rmcol = '005'

site='KUMC'
year=3000

model = pickle.load(open('data/'+site+'/model_'+model_type+'_'+site+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+'.pkl', 'rb'))

#load tables
X_train = pd.read_pickle('data/'+site+'/X_train_'+site+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+'.pkl')
X_test =  pd.read_pickle('data/'+site+'/X_test_' +site+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+'.pkl')
y_train = pd.read_pickle('data/'+site+'/y_train_'+site+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+'.pkl')
y_test =  pd.read_pickle('data/'+site+'/y_test_' +site+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+'.pkl')

pred = model.predict_proba(X_test)

In [ ]:
from sklearn.metrics import roc_curve
from matplotlib import pyplot
roc = roc_auc_score(y_test, pred[:,1])
ns_fpr, ns_tpr, _ = roc_curve(y_test, pred[:,1])
pyplot.plot(ns_fpr, ns_tpr)
# axis labels
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')
# show the plot
pyplot.show()
sklearn.metrics.auc(ns_fpr, ns_tpr)

In [ ]:
from catboost import Pool, cv
import xgboost
import catboost

stg = 'stg01'
fs = 'rmscrbun'
oversample='raw'
model_type = 'catd'    
rmcol = '005'

site='MCRI'
year=3000

model = pickle.load(open('data/'+site+'/model_'+model_type+'_'+site+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+'.pkl', 'rb'))

#load tables
X_train = pd.read_pickle('data/'+site+'/X_train_'+site+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+'.pkl')
X_test =  pd.read_pickle('data/'+site+'/X_test_' +site+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+'.pkl')
y_train = pd.read_pickle('data/'+site+'/y_train_'+site+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+'.pkl')
y_test =  pd.read_pickle('data/'+site+'/y_test_' +site+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+'.pkl')

# Get AUC
#    pred = model.get_booster().predict(dtest, pred_contribs=False)
#    pred = model.predict(X_test)    
#    roc = roc_auc_score(y_test, pred)    

pred = model.predict_proba(X_test)
roc = roc_auc_score(y_test, pred[:,1])    

shapX = pd.concat([X_train, X_test])
shapy = pd.concat([y_train, y_test])

# Calculate SHAP value
if type(model) == xgboost.sklearn.XGBClassifier:
    dshap  = xgb.DMatrix(shapX, label=shapy)
    shap = model.get_booster().predict(dshap, pred_contribs=True)
    # Get feature importance
    model_data = pd.concat([pd.DataFrame(model.get_booster().get_score(importance_type='cover'), index=['Cover']), \
    pd.DataFrame(model.get_booster().get_score(importance_type='gain'), index=['Gain']), \
    pd.DataFrame(model.get_booster().get_score(importance_type='weight'), index=['Frequency'])]).transpose() >> mutate(Feature = X.index)
    model_data['rank'] = model_data['Gain'].rank(method='min', ascending=False)
    used_feature = list(model.get_booster().get_score().keys())        
elif type(model) == catboost.core.CatBoostClassifier:
    cat_features = model.get_cat_feature_indices()
    pshap = Pool(data=shapX, label=shapy, cat_features=cat_features)        
    shap = model.get_feature_importance(data=pshap, type='ShapValues')
    model_data = model.get_feature_importance(prettified=True)
    model_data['Feature'] = model_data['Feature Id']
    model_data = model_data >> select('Feature', 'Importances')
    model_data['rank'] = model_data['Importances'].rank(method='min', ascending=False)     
    used_feature = list((model_data >> mask(X.Importances!=0)).Feature)
else:
#Using shap package example
    import shap
    explainer = shap.Explainer(model, algorithm='permutation')
    shap_valuesX = explainer.shap_values(shapX)
    #shap.summary_plot(shap_valuesX, X_test, plot_type="bar")    
    shap = shap_valuesX    


# Collect SHAP value
def CI95(data):
    if len(data) == 1:
        return (np.nan, np.nan)
    return st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data)) #95% confidence interval

shap_data = list()
shap_data_raw = list()
#for i in range(shapX.columns.shape[0]):
i = 0
df = pd.DataFrame(list(zip(shapX.iloc[:,i], shap[:, i], abs(shap[:, i]))),columns =['Name', 'val', 'absval'])


In [ ]:
df[shapy==1].shape

In [ ]:
plot_data_all = df.groupby("Name").agg([np.mean, np.var, np.std, np.median, CI95, 'size']).reset_index()
plot_data_0= df[shapy==0].groupby("Name").agg([np.mean, np.var, np.std, np.median, CI95, 'size']).reset_index()
plot_data_1= df[shapy==1].groupby("Name").agg([np.mean, np.var, np.std, np.median, CI95, 'size']).reset_index()
plot_data_all.columns = [''.join(x) for x in plot_data_all.columns]
plot_data_0.columns = [x+'_0' for x in plot_data_all.columns]
plot_data_1.columns = [x+'_1' for x in plot_data_all.columns]
plot_data_all = plot_data_all.drop('absvalsize', axis=1)
plot_data_0   = plot_data_0.drop('absvalsize_0', axis=1)
plot_data_1   = plot_data_1.drop('absvalsize_1', axis=1)
plot_data_0 = plot_data_0.rename({'Name_0':'Name'},axis=1)
plot_data_1 = plot_data_1.rename({'Name_1':'Name'},axis=1)
plot_data = pd.merge(plot_data_all, plot_data_0, left_on='Name', right_on='Name', how='left')
plot_data = pd.merge(plot_data, plot_data_1, left_on='Name', right_on='Name', how='left')

In [ ]:
site = 'UofU'
year = 3000
stg = 'stg01'

In [ ]:
data = pd.read_pickle('data/'+site+'/bt3_'+site+'_'+stg+'_3000.pkl')
data.shape

In [ ]:
p0 = data[data['FLAG']==0].select_dtypes('bool').sum()/data[data['FLAG']==0].shape[0]
p1 = data[data['FLAG']==1].select_dtypes('bool').sum()/data[data['FLAG']==1].shape[0]

In [ ]:
dfp = pd.DataFrame([p0,p1]).T